In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from predrain import *

## Preprocessing

Contiguous

In [3]:
%run predrain convert_to_tf_records ../data/SRAD2018/ 1,2 ../data False contiguous --nt 10

Processing subdirectory: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:25<00:00, 197.29it/s]
2it [01:19, 39.76s/it]

In [4]:
%run predrain convert_to_tf_records ../data/SRAD2018/ 1 ../data True contiguous --nt 10

Processing subdirectory: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:50<00:00, 199.23it/s]
1it [00:50, 50.59s/it]

Skip

In [1]:
%run predrain convert_to_tf_records ../data/SRAD2018/ 1,2 ../data False skip --nt=10 --stop=5

Processing subdirectory: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:57<00:00, 87.13it/s]
2it [01:46, 53.28s/it]

In [2]:
%run predrain convert_to_tf_records ../data/SRAD2018/ 1 ../data True skip --nt=10 --stop=5

Processing subdirectory: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3246.21it/s]
1it [00:03,  3.20s/it]

In [17]:
%run predrain generate_stats ../data/SRAD2018/ False 1,2 ../data

5000it [00:00, 224447.96it/s]
5000it [00:00, 261575.07it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:33<00:00, 76.73s/it]

In [2]:
%%time
%run predrain generate_stats ../data/SRAD2018/ True 1 ../data

0it [00:00, ?it/s]

OSError: Error handling image at: ../data/SRAD2018/SRAD2018_Test_1/RAD_206482434252547/RAD_206482434252547_013.png

CPU times: user 11.4 s, sys: 3.63 s, total: 15 s
Wall time: 1min 20s


## Train

In [4]:
predrain = Predrain()

Entry point (after initialization)

In [6]:
# ??predrain.set_flags
??predrain.train

Signature: predrain.train(sz=128, nt=10, bs=12, version=1, num_gpus=1, gpu_start=0)
Docstring: <no docstring>
Source:   
    def train(self, sz=128, nt=10, bs=12, version=1, num_gpus=1, gpu_start=0):
        self.set_flags(sz=sz, nt=nt, bs=bs, version=version, num_gpus=num_gpus, gpu_start=gpu_start)
        
        aug_tfms = self.get_tfms()
        
        model = self.get_model()
        
        trn_tensors, val_tensors = self.get_data()
        
        self.fit()
File:      ~/jupyter/tianchi/pred-rain/yang/predrain.py
Type:      method


### Data

In [10]:
import multiprocessing

# multiprocessing.cpu_count()
print(multiprocessing.cpu_count())

32


In [9]:
??predrain.get_data

Signature: predrain.get_data(base_path=PosixPath('../data/tfrecords'), mode='contiguous', idx_list=[1], val_split=0.1, num_parallel_calls=32, buffer_size=3)
Docstring: <no docstring>
Source:   
    def get_data(self, base_path=Path('../data/tfrecords'), mode='contiguous', idx_list=[1],
                 val_split=0.1, num_parallel_calls=32, buffer_size=3):
        # train_1_contiguous_10
        fn_records = ['train_' + str(idx) + '_' + str(mode) + '_' + str(self.nt) for idx in idx_list]
        fns = [str(Path(base_path)/o) for o in fn_records]
        
        trn_tensors = input_fn(bs, sz, nt, aug_tfms, fns, is_val=False, val_split=val_split,
                                     num_parallel_calls=num_parallel_calls, shuffle=True,
                                     buffer_size=buffer_size)
        val_tensors = input_fn(bs, sz, nt, aug_tfms, fns, is_val=True, val_split=val_split,
                                     num_parallel_calls=num_parallel_calls, shuffle=False,
            

In [11]:
??predrain.input_fn

Signature: predrain.input_fn(bs, sz, nt, aug_tfms, fns, is_val=False, val_split=0.1, stats_sep=',', num_parallel_calls=8, shuffle=False, buffer_size=2, shuffle_buffer_size=24)
Source:   
    def input_fn(self, bs, sz, nt, aug_tfms, fns, is_val=False, val_split=0.1,
                 stats_sep=',', num_parallel_calls=8, 
                 shuffle=False, buffer_size=2, shuffle_buffer_size=24):
        """
        TODO: data interleave
        Create tf.data.Iterator from tfrecord file.

        Receives: 
            fns = a list of tfrecords files **without suffix**
            sz  = the dimension the input images are resized to (int)
            aug_tfms = a list of transforms to apply to input images
        """
        fn_records = [o + '.tfrecords' for o in fns]
        dataset = tf.data.TFRecordDataset(fn_records)

        def parser_train(serialized_example):
            # experimental. TODO: read only needed samples
            shape = (nt, 501, 501, 3)
            context_features

In [7]:
??predrain.get_tfms

Signature: predrain.get_tfms()
Docstring: <no docstring>
Source:   
    def get_tfms(self):
        # data augmentation
        # not implemented yet. see the TODO section in README.md
        # from utils.transforms import *
        aug_tfms = []
        return aug_tfms
File:      ~/jupyter/tianchi/pred-rain/yang/predrain.py
Type:      method


In [8]:
??predrain.get_model

Signature: predrain.get_model(optimizer=<tensorflow.python.keras.optimizers.SGD object at 0x7f07cadd3b38>)
Docstring: <no docstring>
Source:   
    def get_model(self, optimizer=SGD(lr=0.002, momentum=0.9, nesterov=True)):
        # TODO: 
        #   1. add experiments to argument list
        #   2. load models from files
        
        from models.prednet_refactored import PredNetCell, PredNet
        n_channels, im_height, im_width = (1, sz, sz)
        input_shape = (im_height, im_width, n_channels)
        stack_sizes = (n_channels, 48, 96, 192)
        R_stack_sizes = stack_sizes
        A_filt_sizes = (3, 3, 3)
        Ahat_filt_sizes = (3, 3, 3, 3)
        R_filt_sizes = (3, 3, 3, 3)

        layer_loss_weights = np.array([1., 0., 0., 0.])  # weighting for each layer in final loss; 
                                                        # "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
        layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
        time_l

## Evaluation

## Visualization